# TP Final de AA

## Modelo de Rede Neuronais com 20%  

Utilizamos o modelo porque ele é bom com regressão

## Bibliotecas a serem instaladas

In [2]:
import sys
import subprocess

required_packages = ['pandas', 'scikit-learn']
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"{package} não encontrado. Instalando...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])


scikit-learn não encontrado. Instalando...


In [8]:
!pip install matplotlib

In [10]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Modelos
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

print("Todas as bibliotecas foram importadas com sucesso!")

Todas as bibliotecas foram importadas com sucesso!


## Carregamento dos dados

In [11]:
base_dir = "C:/Users/lusca/Universidade/AA/TPFinal/ML-Carros-Usados"
TRAIN = os.path.join(base_dir, "train.csv")
TEST = os.path.join(base_dir, "test.csv")

# Carregar os dados
train = pd.read_csv(TRAIN)
print(train.head())


   id          brand              model  model_year  milage      fuel_type  \
0   0           MINI      Cooper S Base        2007  213000       Gasoline   
1   1        Lincoln              LS V8        2002  143250       Gasoline   
2   2      Chevrolet  Silverado 2500 LT        2002  136731  E85 Flex Fuel   
3   3        Genesis   G90 5.0 Ultimate        2017   19500       Gasoline   
4   4  Mercedes-Benz        Metris Base        2021    7388       Gasoline   

                                              engine  \
0       172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel   
1       252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel   
2  320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...   
3       420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel   
4       208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel   

                     transmission ext_col int_col  \
0                             A/T  Yellow    Gray   
1                             A/T  Silver   Beige   
2                             A/T  

## Pré-Processamento dos dados

In [12]:
# Remover cols inuteis
train = train.drop(['id'], axis=1)

# LabelEncoder para transformar variáveis categóricas em numéricas
categorical_cols = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    label_encoders[col] = le 

X = train.drop(['price'], axis=1) # Features
y = train['price'] # Target

X_train, X_v, y_train, y_v = train_test_split(X, y, test_size=0.2, random_state=7) # 20% para validação


## Treino

In [14]:
model = MLPRegressor(hidden_layer_sizes=(50), max_iter=2000, alpha=0.0001, solver='adam', random_state=7)
model.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=50, max_iter=2000, random_state=7)

## Previsão e análise do treino

In [15]:
y_pred = model.predict(X_v)

# Calcular o MSE
mse = mean_squared_error(y_v, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.2f} euros")
print(f"MAE: {mean_absolute_error(y_v, y_pred):.2f} euros")
print(f"RRMSE: {rmse / y.mean() * 100:.2f}%")
print(f"Media dos precos: {y.mean():.2f} euros")
print(f"Mediana dos precos: {y.median():.2f} euros")
print(f"R2: {r2_score(y_v, y_pred):.2f}")

RMSE: 69596.12 euros
MAE: 20586.98 euros
RRMSE: 158.61%
Media dos precos: 43878.02 euros
Mediana dos precos: 30825.00 euros
R2: 0.14


## Pré-Processamento dos dados de teste

In [16]:
test = pd.read_csv(TEST)

# Aplicar o mesmo método do treino no teste
for col in categorical_cols:
    # Adicionar "unknown" aos encoders para lidar com valores não vistos
    label_encoders[col].classes_ = np.append(label_encoders[col].classes_, 'unknown')
    
    # Substituir valores não vistos por "unknown" e aplicar a transformação
    test[col] = test[col].apply(lambda x: x if x in label_encoders[col].classes_ else 'unknown')
    test[col] = label_encoders[col].transform(test[col])

## Teste e criação do CSV

In [27]:
test_predictions = model.predict(test.drop(['id'], axis=1))

submission = pd.DataFrame({
    'id': test['id'],           # IDs originais do conjunto de teste
    'price': test_predictions   # Previsões feitas pelo modelo
})

submission_file = os.path.join(base_dir, 'submission.csv')
submission.to_csv(submission_file, index=False)
print(f"Arquivo de submissão gerado com sucesso: {submission_file}")


Arquivo de submissão gerado com sucesso: C:/Users/lusca/Universidade/AA/TPFinal/ML-Carros-Usados\submission.csv


## Melhor Submissão deste Jupyter a utilizar Redes Neuronais 

![Submissão](./Captura.png)

## Comparação com outros modelos

Primeiro, setando os modelos

In [17]:
RandomForest = RandomForestRegressor(n_estimators=100, random_state=7)
SVRmodel = SVR(kernel='rbf')
GBR = GradientBoostingRegressor(n_estimators=500, random_state=7)

models = {
    "Random Forest": RandomForest,
    "SVR": SVRmodel,
    "Gradient Boosting": GBR
}



## Executando modelo por modelo

In [ ]:
results = {"Model": [], "RMSE": [], "R2 Score": []}

for name, model in models.items():
    # Treinamento
    model.fit(X_train, y_train)
    
    # Previsões
    y_pred = model.predict(X_v)
    
    # Calculando métricas
    rmse = np.sqrt(mean_squared_error(y_v, y_pred))
    r2 = r2_score(y_v, y_pred)
    
    # Armazenar os resultados
    results["Model"].append(name)
    results["RMSE"].append(rmse)
    results["R2 Score"].append(r2)



## Estudo dos resultados

In [ ]:
results_df = pd.DataFrame(results)
plt.figure(figsize=(12, 6))

# RMSE
plt.subplot(1, 2, 1)
plt.bar(results_df["Model"], results_df["RMSE"], color='skyblue')
plt.title("Comparação de RMSE")
plt.ylabel("RMSE")
plt.xticks(rotation=45)

# R2 Score
plt.subplot(1, 2, 2)
plt.bar(results_df["Model"], results_df["R2 Score"], color='lightgreen')
plt.title("Comparação de R2 Score")
plt.ylabel("R2 Score")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()